## Final step -- search data for keywords

In [27]:
import pandas as pd
import string

In [38]:
lennar = pd.read_json('./lennar-data-06-2023.json')

In [39]:
lennar.shape

(10081, 180)

In [40]:
lennar.head(1)

,Availability,Price,Beds,Baths,Sqft,Address,Community,URL,Data_Dump,feature_0,...,feature_161,feature_162,feature_163,feature_164,feature_165,feature_166,feature_167,feature_168,feature_169,feature_170
0,Move-in ready,"$347,815",3.0,3.5,1744.0,"6235 Truxton Lane, Raleigh, NC",Manteo II in 5401 North,https://www.lennar.com/new-homes/north-carolin...,"{'Kitchen': ['36"" cabinetry with crown molding...","36"" cabinetry with crown molding and nickel ha...",...,None,None,None,None,None,None,None,None,None,None


In [41]:
clean = lennar.drop_duplicates(subset=['Availability', 'Price', 'Beds', 'Baths', 'Sqft', 'Address', 'Community', 'URL'],
                    keep='first',
                    ignore_index=True)

In [42]:
clean.shape

(8859, 180)

After removing duplicate homes (from the fact that some cities that are close together list the same homes under different urls -- ex Charlotte is under SC and NC for some reason), we see there are **9716 homes in the dataset**

In [43]:
# import re

In [44]:
# for d in clean['Data_Dump'][500:]:
#     lines = [item.lower() for sublist in d.values() for item in sublist]
    
#     # get rid of characters we don't want
#     cleaned_lines = [l.strip().replace('\n', '').replace('\r', '').replace('®', '').
#                      replace('™','').replace('¹', '').replace('”','') for l in lines]
    
#     # remove punc other than dash
#     ultra_cleaned_lines = [cl.translate(str.maketrans('', '', string.punctuation.replace('-',''))) 
#                            for cl in cleaned_lines]
    
#     cleaned_words = [i for l in ultra_cleaned_lines for i in l.split()]
    
#     print(cleaned_words)
#     break

In [69]:
def search_homes(date, keyword, save=False):
    """
    Method that searches through the Data_dump dict structure
    
    Returns:
        vals: dict where keys include keyword and values are counts
        home_count: # of unique homes where the keyword is present
    """
    lennar = pd.read_json(f'./lennar-data-{date}.json')
    clean = lennar.drop_duplicates(subset=['Availability', 'Price', 'Beds', 'Baths', 'Sqft', 'Address', 'Community', 'URL'],
                    keep='first',
                    ignore_index=True)
    
    keyword = keyword.lower()
    vals = dict()
    home_count = 0
    index = 0
    
    for d in clean.Data_Dump:
        index += 1
        lines = [item.lower() for sublist in d.values() for item in sublist]
    
        # get rid of characters we don't want
        cleaned_lines = [l.strip().replace('\n', '').replace('\r', '').replace('®', '').
                         replace('™','').replace('¹', '').replace('”','') for l in lines]

        # remove punc other than dash
        ultra_cleaned_lines = [cl.translate(str.maketrans('', '', string.punctuation.replace('-',''))) 
                               for cl in cleaned_lines]

        cleaned_words = [i for l in ultra_cleaned_lines for i in l.split()]
        
        if keyword in cleaned_words:
            home_count += 1
            
            for ucl in ultra_cleaned_lines:
                if keyword in ucl:
                    vals[ucl] = vals.get(ucl, 0) + 1
    
    df = pd.DataFrame([[home_count, sorted(vals.items(), key=lambda x: x[1], reverse=True), 
                        sum(vals.values())]], 
                      columns = ['n_homes', 'data', 'sum_data_vals'])
    
    if save:
        df.to_csv(f'./search-results-{date}/{keyword}.csv', index=False)
    return pd.DataFrame(df['data'][0], columns=['description', 'count'])

Okay so here's the function. Try 'usb', 'wifi', 'motion' etc

In [102]:
june_df = search_homes('06-2023', 'ring')
oct_df = search_homes('10-2022', 'ring')

In [103]:
june_df.head()

,description,count
0,ring video doorbell pro,3639
1,ring alarm security kit – base station,2293
2,ring alarm security kit – ring alarm keypad,1917
3,ring alarm security kit – ring alarm motion de...,1917
4,ring alarm security kit – ring alarm smoke and...,1917


In [104]:
oct_df.head()

,description,count
0,ring video doorbell pro,6304
1,ring alarm security kit – base station,4305
2,ring alarm security kit – ring alarm motion de...,3891
3,ring alarm security kit – ring alarm smoke and...,3860
4,ring alarm security kit – ring alarm keypad,3768


In [105]:
def percentage_change(col1,col2):
    return round(((col2 - col1) / col1) * 100, 2)

In [106]:
df = june_df.merge(oct_df, how='outer', on='description', suffixes=('_06-23','_10-22'))

In [107]:
df.fillna(0, inplace=True)
sorted_indices = (df["count_06-23"] + df["count_10-22"]).sort_values(ascending=False).index
df = df.loc[sorted_indices, :]
df['pct-change'] = percentage_change(df['count_10-22'],df['count_06-23'])    

In [108]:
df

,description,count_06-23,count_10-22,pct-change
0,ring video doorbell pro,3639.0,6304.0,-42.27
1,ring alarm security kit – base station,2293.0,4305.0,-46.74
3,ring alarm security kit – ring alarm motion de...,1917.0,3891.0,-50.73
4,ring alarm security kit – ring alarm smoke and...,1917.0,3860.0,-50.34
2,ring alarm security kit – ring alarm keypad,1917.0,3768.0,-49.12
...,...,...,...,...
291,ring alarm security kit with base station keyp...,1.0,0.0,inf
292,osb subflooring,1.0,0.0,inf
518,stylish and durable flooring in the kitchen di...,0.0,1.0,-100.00
519,shawspan stylecolor 2d2d2dregspannbspluxury vi...,0.0,1.0,-100.00


In [109]:
df.to_csv('./analysis/ring-description-differences.csv', index=False)